# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
df = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
df.head(5)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [9]:
df.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

# What test is appropriate for this problem? Does CLT apply?
A two-sampled z-test for proportions would be appropriate for this problem, and the Central Limit Theorem would apply provided four conditions hold true:
1. The data was collected as a simple random sample of the population.
2. Each observation is independent of other observations in the sample.
3. For each sample, there are at least 10 successes and at least 10 failures.
4. The population from which the sample is collected is at least 20 times greater than the sample.

Condition 1 is given by the problem statement. Condition 2 is implied by Condition 1. Condition 4 can be safely assumed. Condition 3 is satisifed as evidenced by the chart below.

In [10]:
df.groupby(['race','call']).size().unstack('race')

race,b,w
call,,
0.0,2278,2200
1.0,157,235


# What are the null and alternate hypotheses?

Let $p_b$ represent the proportion of resumes with black names that received callbacks, and let $p_w$ represent the proportion of resumes with white names that received callbacks.

$H_0: p_b - p_w = 0 \\
H_A: p_b - p_w < 0$

# Calculate the margin of error, confidence interval, and p-value.

In [9]:
n_b = len(df[df.race == 'b']) # number of resumes with black names
n_w = len(df[df.race == 'w']) # number of resumes with white names
x_b = sum(df[df.race == 'b'].call) # number of resumes with black names that received callbacks
x_w = sum(df[df.race == 'w'].call) # number of resumes with black names that received callbacks
p_b = x_b / n_b # proportion of resumes with black names that received callbacks
p_w = x_w / n_w # proportion of resumes with white names that received callbacks
v_b = p_b * (1 - p_b) / n_b # variance black resumes
v_w = p_w * (1 - p_w) / n_w # variance white resumes

# margin of error = critical value * standard error
# confidence interval = point estimate +/- margin of error
# confidence level = 0.95

point_estimate = p_b - p_w
standard_error = np.sqrt(v_b + v_w)
margin_of_error = stats.norm.ppf(.975) * standard_error
confidence_interval = point_estimate + np.array([-1,1]) * margin_of_error

# hypothesis test
z_statistic = point_estimate / standard_error
p_value = stats.norm.cdf(z_statistic)

print('point estimate:',point_estimate)
print('confidence_interval:',confidence_interval)
print('margin of error:',margin_of_error)
print('p-value:',p_value)

point estimate: -0.0320328542094
confidence_interval: [-0.04728798 -0.01677773]
margin of error: 0.0152551260282
p-value: 1.93128260376e-05


# Write a story describing the statistical significance in the context or the original problem.
In the sample data collected, resumes with black applicant names received 3.2% fewer callbacks than those with white applicant names. To check if this difference was statistically significant, a hypothesis test with a 95% confidence level was conducted, resulting in a p-value of $1.93\times10^{-5} < .05$. Thus, it can be claimed with 95% confidence that resumes with black applicant names do in fact experience lower rates of callback success.

# Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
While a statistically significant difference in callback rates between resumes with black and white applicant names was detected, the data does not provide a basis for claiming that race/name is the most important factor in callback success. One would have to check each variable to see if differentiation by that attribute results in a greater disparity in callback rates so as to account for the possibility of confounding factors.